In [13]:
# install and load packages
import pandas as pd
import mechanize
from http.cookiejar import CookieJar
from bs4 import BeautifulSoup as soup
import html2text

In [6]:
# Load xlsx dataset
Fixed = pd.ExcelFile("..//Datasets/FW/FixedWing_021021_(modified).xlsx", engine='openpyxl')

In [3]:
# excel content
Fixed.sheet_names

['Sheet1']

In [301]:
# Extract "Data" sheet
FW = Fixed.parse(0)

# Extract tail numbers
tails = FW.Tail_number.to_list()
len(tails)

210

In [5]:
# login to the website
# Browser
br = mechanize.Browser()

# Cookie Jar
cj = CookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]

# The site we will navigate into, handling it's session
br.open('https://flightaware.com/account/session')

# View available forms
for f in br.forms():
    print (f)

# Select the second (index one) form (the first form is a search query box)
br.select_form(nr=7)

# User credentials
br.form["flightaware_username"] = 'kismailo'
br.form['flightaware_password'] = 'gwynbleidd'

# Login
br.submit()

<get https://flightaware.com/account/locale_switch.rvt application/x-www-form-urlencoded
  <SelectControl(locale=[ar_SA, cs_CZ, de_DE, en_AU, en_CA, en_GB, *en_US, es_ES, es_MX, fi_FI, fr_CA, fr_FR, he_IL, hi_IN, hr_HR, it_IT, ja_JP, ko_KR, pl_PL, pt_BR, pt_PT, ru_RU, tr_TR, zh_CN, zh_HK, zh_TW])>>
<get https://flightaware.com/account/locale_switch.rvt application/x-www-form-urlencoded
  <SelectControl(locale=[ar_SA, cs_CZ, de_DE, en_AU, en_CA, en_GB, *en_US, es_ES, es_MX, fi_FI, fr_CA, fr_FR, he_IL, hi_IN, hr_HR, it_IT, ja_JP, ko_KR, pl_PL, pt_BR, pt_PT, ru_RU, tr_TR, zh_CN, zh_HK, zh_TW])>>
<get https://flightaware.com/ajax/ignoreall/omnisearch/disambiguation.rvt application/x-www-form-urlencoded
  <HiddenControl(searchterm=) (readonly)>
  <HiddenControl(token=) (readonly)>
  <HiddenControl(<None>=) (readonly)>>
<GET https://flightaware.com/live/findflight application/x-www-form-urlencoded
  <TextControl(origin=)>
  <TextControl(destination=)>
  <SubmitControl(<None>=) (readonly)>>
<

<response_seek_wrapper at 0x7f96dd698750 whose wrapped object = <closeable_response at 0x7f96de10cf10 whose fp = <mechanize._gzip.UnzipWrapper object at 0x7f96ddf19710>>>

In [327]:
tail_number = []
date = []
aircraft = []
origin = []
origin_location = []
destination = []
destination_location = []
departure = []
arrival = []
duration = []

for t in tails:
    url = 'https://flightaware.com/live/flight/' + t + '/history/320'
    page_html = br.open(url).read()
    page_soup = soup(page_html, 'html.parser')
    table = page_soup.findAll("tr", {"class":["smallActiverow1 rowClickTarget", "smallActiverow2 rowClickTarget"]})

    for i in range(0,len(table)):
        tail_number += [t]
        date += table[i].find_all("td")[0].a
        aircraft += [table[i].find_all("td")[1].text]
        origin += [table[i].find_all("td")[2].text]
        if len(table[i].find_all("td")[2].text) != 0:
            origin_location += [table[i].find_all("td")[2].span['title']]
        else:
            origin_location += ["Unknown"]
        destination += [table[i].find_all("td")[3].text]
        if len(table[i].find_all("td")[3].text) != 0:
            destination_location += [table[i].find_all("td")[3].span['title']]
        else:
            destination_location += ["Unknown"]
        departure += [table[i].find_all("td")[4].text]
        arrival += [table[i].find_all("td")[5].text]
        duration += [table[i].find_all("td")[6].text]  
            
data = pd.DataFrame(list(zip(tail_number, date, aircraft, origin, origin_location, destination, destination_location, departure, arrival, duration)), 
                    columns=['tail_number','date', 'aircraft', 'origin', 'origin_location', 'destination', 'destination_location', 'departure', 'arrival', 'duration'])

In [343]:
data

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,arrival,duration
0,N7025P,28-Feb-2021,Unknown,"Near Bend, OR",L 43.95194 -121.28694,,Unknown,First seen 04:59PM PST,,En Route
1,N7025P,28-Feb-2021,Unknown,Tews Fld (CA53),"Tews Fld (Redding, CA) - CA53","Near Bend, OR",L 43.81944 -121.37583,01:36PM PST,Last seen 03:01PM PST,1:24
2,N7025P,21-Feb-2021,Unknown,"Near Lincoln, CA",L 38.93333 -121.41667,Redding Muni (KRDD),"Redding Muni (Redding, CA) - KRDD",First seen 03:02PM PST,Last seen 03:49PM PST,0:47
3,N7025P,21-Feb-2021,Unknown,"Near Red Bluff, CA",L 40.24000 -122.19083,"Near Marysville, CA",L 38.94750 -121.52250,First seen 12:46PM PST,Last seen 01:20PM PST,0:33
4,N7025P,24-Dec-2020,Unknown,"Near Red Bluff, CA",L 40.28861 -122.05861,"Near Emigrant Gap, CA",L 39.77972 -120.52333,First seen 09:25AM PST,Last seen 10:02AM PST,0:36
...,...,...,...,...,...,...,...,...,...,...
11236,N977TC,30-Nov-2020,C680,El Paso Intl (KELP),"El Paso Intl (El Paso, TX) - KELP",William P Hobby (KHOU),"William P Hobby (Houston, TX) - KHOU",03:30PM MST,05:54PM CST,1:23
11237,N977TC,30-Nov-2020,C680,William P Hobby (KHOU),"William P Hobby (Houston, TX) - KHOU",El Paso Intl (KELP),"El Paso Intl (El Paso, TX) - KELP",11:11AM CST,11:52AM MST,1:40
11238,N102WK,12-Feb-2021,BE9T,Smyrna (KMQY),"Smyrna (Smyrna, TN) - KMQY",Shreveport Rgnl (KSHV),"Shreveport Rgnl (Shreveport, LA) - KSHV",11:19AM CST,01:34PM CST,2:14
11239,N102WK,28-Jan-2021,BE9T,Shreveport Rgnl (KSHV),"Shreveport Rgnl (Shreveport, LA) - KSHV",Smyrna (KMQY),"Smyrna (Smyrna, TN) - KMQY",12:30PM CST,02:25PM CST,1:55


In [348]:
# merge names, and export data
data.merge(FW[['Tail_number','NAME']], left_on='tail_number', right_on='Tail_number', how='inner').drop('Tail_number', axis=1).rename({'NAME':'medical_service'}, axis=1)\
    .to_csv('..//Datasets/FW/FW_tail_numbers.csv', index=False)

In [296]:
# # tail number web scraping
# def tail_numbers(tail):
#     tail_number = []
#     date = []
#     aircraft = []
#     origin = []
#     origin_location = []
#     destination = []
#     destination_location = []
#     departure = []
#     arrival = []
#     duration = []

#     for t in tail:
#         url = 'https://flightaware.com/live/flight/' + t + '/history/320'
#         page_html = br.open(url).read()
#         page_soup = soup(page_html, 'html.parser')
#         table = page_soup.findAll("tr", {"class":["smallActiverow1 rowClickTarget", "smallActiverow2 rowClickTarget"]})

#         for i in range(0,len(table)):
#             tail_number += [t]
#             date += table[i].find_all("td")[0].a
#             aircraft += [table[i].find_all("td")[1].text]
#             origin += [table[i].find_all("td")[2].text]
#             origin_location += [table[i].find_all("td")[2].span['title']]
#             destination += [table[3].find_all("td")[3].text]
#             destination_location += [table[i].find_all("td")[3].span['title']]
#             departure += [table[i].find_all("td")[4].text]
#             arrival += [table[i].find_all("td")[5].text]
#             duration += [table[i].find_all("td")[6].text]  
            
#             date += table[i].find_all("td")[0].a
#             aircraft += table[i].find_all("td")[1]
#             origin += table[i].find_all("td")[2].span
#             if len(table[i].find_all("td")[3]) != 0:
#                 destination += table[i].find_all("td")[3].span
#             if len(table[i].find_all("td")[3]) == 0:
#                 destination += ["Unknown"]
#             if len(table[i].find_all("td")[4]) == 1:
#                 departure += [table[i].find_all("td")[4].em]
#             if len(table[i].find_all("td")[4]) == 2:
#                 departure += [table[i].find_all("td")[4]]
#             if table[i].find_all("td")[5].em != 'NoneType':
#                 arrival += [table[i].find_all("td")[5]]
#             if table[i].find_all("td")[5].em == 'NoneType':
#                 arrival += table[i].find_all("td")[5].em
#             duration += table[i].find_all("td")[6]

#     data = pd.DataFrame(list(zip(tail_number, dates, aircraft, origin, origin_location, destination, destination_location, departure, arrival, duration)), 
#                     columns=['tailnumber','dates', 'aircraft', 'origin', 'origin_location', 'destination', 'destination_location', 'departure', 'arrival', 'duration'])
#     return(data)

In [ ]:
# # apply function
# data = tail_numbers(tails)